In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 9.3 MB/s eta 0:00:00ta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
dataset = pd.read_csv("/kaggle/input/postagger/POS.csv")

In [5]:
import datasets
datasets.__version__

'4.1.1'

In [6]:
dataset.head()

,sentence_id,words,labels
0,0,﻿६१,CD
1,0,वर्षीय,JJ
2,0,पियरे,NNP
3,0,भिन्केन,NNP
4,0,नोभेम्बर,NNP


In [7]:
label_list = dataset['labels'].unique().tolist()
print(len(label_list))

39


In [8]:
label2ids = {label:i for i,label in enumerate(label_list)}
ids2label = {values:keys for keys,values in label2ids.items()}

In [9]:
print(label2ids)
print(ids2label)

{'CD': 0, 'JJ': 1, 'NNP': 2, 'POP': 3, 'NN': 4, 'PKO': 5, 'VBX': 6, 'YF': 7, 'FB': 8, 'VBF': 9, 'PLAI': 10, 'DUM': 11, 'VBKO': 12, 'RBO': 13, 'VBI': 14, 'VBO': 15, 'HRU': 16, 'JJD': 17, 'YM': 18, 'PLE': 19, 'JJM': 20, 'RP': 21, 'VBNE': 22, 'CS': 23, 'YQ': 24, 'CL': 25, 'PP': 26, 'PP$': 27, 'CC': 28, 'SYM': 29, 'PPR': 30, 'DM': 31, 'OD': 32, 'QW': 33, 'UNW': 34, 'RBM': 35, 'FW': 36, 'YB': 37, 'ALPH': 38}
{0: 'CD', 1: 'JJ', 2: 'NNP', 3: 'POP', 4: 'NN', 5: 'PKO', 6: 'VBX', 7: 'YF', 8: 'FB', 9: 'VBF', 10: 'PLAI', 11: 'DUM', 12: 'VBKO', 13: 'RBO', 14: 'VBI', 15: 'VBO', 16: 'HRU', 17: 'JJD', 18: 'YM', 19: 'PLE', 20: 'JJM', 21: 'RP', 22: 'VBNE', 23: 'CS', 24: 'YQ', 25: 'CL', 26: 'PP', 27: 'PP$', 28: 'CC', 29: 'SYM', 30: 'PPR', 31: 'DM', 32: 'OD', 33: 'QW', 34: 'UNW', 35: 'RBM', 36: 'FW', 37: 'YB', 38: 'ALPH'}


In [10]:
dataset.isna().sum()

sentence_id    0
words          2
labels         0
dtype: int64

In [11]:
index = dataset[dataset["words"].isna()].index.tolist()
print(index)

[61652, 73919]


In [12]:
dataset = dataset.drop(index=index)

In [13]:
print(len(dataset["sentence_id"].unique()))

4251


In [14]:
grouped = dataset.groupby("sentence_id")
print(len(grouped["words"]))

4251


In [15]:
df_grouped = grouped.agg({
    "words":lambda x:list(x),
    "labels":lambda x:list(x)
})

In [16]:
df_grouped.columns

Index(['words', 'labels'], dtype='object')

In [17]:
train,test = train_test_split(
    df_grouped,
    test_size = 0.2,
    shuffle = True
)

In [18]:
train.reset_index(drop=True,inplace=True)

In [19]:
train.head()

,words,labels
0,"[यो, कुनै, स्वस्थ, कुरा, होइन, ;, यस, मा, कुनै...","[DUM, DUM, JJ, NN, VBX, YM, DUM, POP, DUM, JJ,..."
1,"[स्वचालित, सेयर, व्यवसायी, हरू, यो, नियम, को, ...","[JJ, NN, NN, HRU, DUM, NN, PKO, NN, PLE, JJ, N..."
2,"[यएएल, को, घोषणा, हिजो, बजार, बन्द, भए, पछि, आ...","[NNP, PKO, NN, RBO, NN, JJ, VBO, POP, VBF, YF]"
3,"[फर्स्ट, कन्सटिच्युसन, ले, डब्ल्युएफआरआर, एल.प...","[NNP, NN, PLE, NNP, FB, CC, FB, NN, POP, CD, N..."
4,"[संघीय, न्यायधिस, हरू, ले, वार्षिक, ८९,५००, डल...","[JJ, NN, HRU, PLE, JJ, CD, NNP, VBF, YM, NNP, ..."


In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")


config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [21]:
print(train['words'].dtype)


object


In [22]:
print(type(train["words"]))

<class 'pandas.core.series.Series'>


In [23]:
tokenized_input = tokenizer(train.iloc[0]["words"],is_split_into_words=True)

In [24]:
print(tokenized_input.keys())

KeysView({'input_ids': [2, 1786, 19867, 10580, 1010, 1041, 7533, 2427, 31, 1782, 1723, 19867, 16210, 5213, 3073, 2650, 428, 601, 601, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})


In [25]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [26]:
word_ids = tokenized_input.word_ids(batch_index=0) 
print(word_ids)

[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11, 12, 13, 13, None]


In [27]:
print(tokens)

['[CLS]', 'यो', 'कन', 'सव', '##स', '##थ', 'करा', 'होइन', ';', 'यस', 'मा', 'कन', 'सवा', '##भाविक', 'गण', 'छन', '।', '‘', '‘', '[SEP]']


In [28]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [29]:
print(type(dataset_train['words']))              # datasets.arrow_dataset.Column
print(type(dataset_train['words'][0]))           # list
print(type(dataset_train['words'][0][0]))        # str
print(type(dataset_train['labels'][0]))          # list
print(type(dataset_train['labels'][0][0]))       # str or int depending on mapping


<class 'datasets.arrow_dataset.Column'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'str'>


In [30]:
tokenized_input = tokenizer(dataset_train["words"][:100],is_split_into_words=True)

In [31]:
# print(dataset_train['words'][0])
# for i,data in enumerate(dataset_train['labels'][0]):
#     print(f"{i}:{data}")
#     print(label2ids[data])
    
 

In [32]:
def tokenize_and_align_labels(examples):
    tokenize_inputs = tokenizer(examples['words'][:],truncation=True,max_length = 512,is_split_into_words=True)
    labels = []

    for i,label in enumerate(examples['labels'][:]):
        word_ids = tokenize_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2ids[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenize_inputs['labels'] = labels
    return tokenize_inputs
                
        
    

In [33]:

tokenized_inputs = dataset_train.map(tokenize_and_align_labels,batched=True,batch_size=100)

Map:   0%|          | 0/3400 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorForTokenClassification

datacollator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2025-10-17 07:10:37.013201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760685037.432449      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760685037.566904      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [37]:
pip install -q evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 43.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*,

In [38]:
import evaluate

seqeval = evaluate.load("seqeval")

Now creating a function that passes predicition and true label to compute metrics


In [41]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p    
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ ids2label[p] for (p,l) in zip(prediction, label) if l != -100 ]
        for prediction,label in zip(predictions,labels) 
    ]

    true_label = [
        [ ids2label[l] for (p,l) in zip(prediction, label) if l != -100 ]
        for prediction,label in zip(predictions,labels) 
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_label)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }